In [1]:
# %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
# import scipy.stats as stats
import scipy.cluster.hierarchy as sch

# from Bio import SeqIO
# from Bio import SeqFeature as sf
# from Bio.Alphabet import generic_dna
# from Bio.Seq import Seq

# import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')


what computer are you on? a = Bens, b = gpucluster, c = other   b
base directory is /home/heineike/github/expression_broad_data
Added /home/heineike/github/expression_broad_data to path: 
['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/local/lib/python3.5/dist-packages/cmonkey2-1.2.6-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/SQLAlchemy_Utils-0.32.14-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/svgwrite-1.1.11-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/Routes-2.4.1-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/CherryPy-10.2.2-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/biopython-1.69-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/configparser-3.5.0-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/rpy2-2.8.6-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/repoze.lru-0.6-py3.5.egg', '

## Load Data

In [2]:
#Read in the KL promoter database.  
kl_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("kl_promoters/kl_promoters.pkl"))

#Read in the SC promoter database.  
sc_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("sc_promoters/sc_promoters.pkl"))

In [2]:
# Load goi data from goi data file created in paralog analysis script. 
goi_data = pd.read_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data_solis.csv")

## Build enrichment sets

Did enrichment analysis using DREME, MEME, and GLAM2 on the upregulated set.  

Only found Msn2 and a few others 


In [21]:
a = '2.0'

In [27]:
#Upregulated v.s. all
LFC_cutoff_high = 2.0
genes = goi_data[goi_data['log2FoldChange_SC']>LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_gt_' + str(LFC_cutoff_high).replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)

#Downregulated v.s. all
LFC_cutoff_low = -2.0
genes = goi_data[goi_data['log2FoldChange_SC']<LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_lt_' + str(LFC_cutoff_low).replace('-','m').replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)



#Upregulated v.s. flat and downregulated
#downregulated v.s. flat and upregulated

all_sc_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/all_sc_promoters.fasta')

YIL111W not in promoter data set.


## Run MEME to look for unbiased enrichment

In [ ]:
#Right now meme doesn't do discriminatory and allow any number of motifs to be scanned. 
#Dreme seems to be better for that. 
#Ran GLAM2 and got a bunch of polyA sites. 


#run psp-gen to get prior for positive and negative sequences. 
psp_gen_command = ['psp-gen', '-pos', target_promoter_fname, '-neg', all_sc_promoter_fname, '-dna', 
                   '-minw', '6', #minimum width of motif
                   '-maxw', '20', #maximum width of motif
                   '>', 'psp_prior.psp']
subprocess.run(psp_gen_command)

In [ ]:
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17358 -maxsize 60000 -mod anr -nmotifs 5 -minw 6 -maxw 50 -revcomp -psp priors.psp
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17402 -maxsize 60000 -mod zoops -nmotifs 3 -minw 6 -maxw 50 -revcomp -psp priors.psp
dreme dreme -v 1 -oc . -dna -p paralogs_LFC_gt_2p0.fasta -n all_sc_promoters.fasta -t 18000 -e 0.05

## Look for Msn2 motifs 

In [10]:
io_library.SC_orf_lookup_by_name(['HXT5'])

['YHR096C']

In [23]:

goi_data_msn2binding

,Unnamed: 0,Unnamed: 0.1,SC_common_name,sc_genename,log2FoldChange_SC,raw_AS_noNMPP1_YPD_t40_mean_SC,pvalue_SC,raw_AS_yesNMPP1_YPD_t40_mean_SC,kl_genename,log2FoldChange_KL,raw_AS_noNMPP1_YPD_t40_mean_KL,raw_AS_yesNMPP1_YPD_t40_mean_KL,pvalue_KL,category,log2FoldChange_msn24del_interaction,pvalue_msn24del_interaction,MSN2_roy_score,MSN4_roy_score
0,0,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,-0.032762,0.992734,1.04961,1.04961
1,0,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,-0.032762,0.992734,1.04961,1.00000
2,0,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,-0.032762,0.992734,1.00000,1.04961
3,0,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,-0.032762,0.992734,1.00000,1.00000
4,1,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,0.686233,0.775722,0.50000,0.50000
5,1,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,0.686233,0.775722,0.50000,0.50000
6,1,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,0.686233,0.775722,0.50000,0.50000
7,1,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,0.686233,0.775722,0.50000,0.50000
8,2,3972,SKT5,YBL061C,-1.806729,2.460149,3.018311e-07,2.056057,KLLA0E20263g,-0.380750,2.462343,2.466842,3.827839e-01,metabolic,0.726361,0.667417,0.00000,0.00000
9,3,3973,SHC1,YER096W,3.793451,1.471903,2.452936e-16,2.754438,KLLA0E20263g,-0.380750,2.462343,2.466842,3.827839e-01,metabolic,-3.095531,0.317884,1.00000,1.00000


In [42]:
goi_data_msn2binding

,Unnamed: 0,Unnamed: 0.1,SC_common_name,sc_genename,log2FoldChange_SC,raw_AS_noNMPP1_YPD_t40_mean_SC,pvalue_SC,raw_AS_yesNMPP1_YPD_t40_mean_SC,kl_genename,log2FoldChange_KL,raw_AS_noNMPP1_YPD_t40_mean_KL,raw_AS_yesNMPP1_YPD_t40_mean_KL,pvalue_KL,category,log2FoldChange_msn24del_interaction,pvalue_msn24del_interaction,MSN2_roy_score,MSN4_roy_score
0,0,3968,GLK1,YCL040W,4.160390,1.791886,3.592258e-04,3.224448,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,-0.032762,0.992734,1.049610,1.0
1,1,3969,EMI2,YDR516C,2.464130,3.271367,1.357405e-16,4.148126,KLLA0C01155g,2.178913,2.557021,3.350059,8.942805e-06,metabolic,0.686233,0.775722,0.500000,0.5
2,2,3972,SKT5,YBL061C,-1.806729,2.460149,3.018311e-07,2.056057,KLLA0E20263g,-0.380750,2.462343,2.466842,3.827839e-01,metabolic,0.726361,0.667417,0.000000,0.0
3,3,3973,SHC1,YER096W,3.793451,1.471903,2.452936e-16,2.754438,KLLA0E20263g,-0.380750,2.462343,2.466842,3.827839e-01,metabolic,-3.095531,0.317884,0.500000,1.0
4,4,3978,GND2,YGR256W,7.143522,1.412570,3.201154e-118,3.676099,KLLA0A09339g,0.394234,3.570254,3.789341,2.751345e-01,metabolic,-4.432623,0.281333,1.389110,1.0
5,5,3979,GND1,YHR183W,-1.190240,2.855015,1.105005e-02,2.639284,KLLA0A09339g,0.394234,3.570254,3.789341,2.751345e-01,metabolic,2.279337,0.243429,0.000000,0.0
6,6,3994,LPD1,YFL018C,1.016738,3.100750,1.818119e-04,3.545507,KLLA0D11154g,-0.817035,2.686856,2.538857,2.828473e-02,metabolic,-0.212748,0.857728,0.000000,0.0
7,7,3995,IRC15,YPL017C,3.049381,0.638741,4.554883e-07,1.593980,KLLA0D11154g,-0.817035,2.686856,2.538857,2.828473e-02,metabolic,-2.339927,0.297505,0.000000,0.5
8,8,4018,ENO1,YGR254W,2.299404,3.536551,3.641904e-05,4.432957,KLLA0A09185g,0.559521,4.339191,4.637226,2.224845e-01,metabolic,1.626104,0.494498,1.389110,1.0
9,9,4019,ENO2,YHR174W,-0.975982,5.013215,1.765184e-04,4.854047,KLLA0A09185g,0.559521,4.339191,4.637226,2.224845e-01,metabolic,1.440190,0.307476,0.000000,0.0


In [48]:
goi_data_msn2binding.sort_values?

In [50]:
#Motif Prior from Roy et al: 

#HSP12: YFL014W
#Msn2: YMR037C
motif_prior = pd.read_table(data_processing_dir + os.path.normpath("/motifs/Motif_prior.txt"), header = None, names = ["TF","sc_genename","interaction_score"])

#shrink motif_prior to include only genes that are included on my subset list
motif_prior_gois = motif_prior[motif_prior["sc_genename"].isin(goi_data['sc_genename'])]
goi_data_msn2binding = goi_data.copy()
goi_data_msn2binding.sort_values('log2FoldChange_msn24del_interaction', axis = 0, inplace=True)

for tf in ['MSN2', 'MSN4']:
    tf_hits = motif_prior_gois[motif_prior_gois['TF'].isin(io_library.SC_orf_lookup_by_name([tf]))]
    goi_data_msn2binding = goi_data_msn2binding.merge(tf_hits.loc[:,['sc_genename','interaction_score']],how = 'left', on = 'sc_genename')
    #Replace NAN's
    goi_data_msn2binding['interaction_score'] = goi_data_msn2binding['interaction_score'].fillna(0)
    goi_data_msn2binding.rename(columns = {'interaction_score': tf + '_roy_score'}, inplace = True)




#Extract only Msn2 hits from Motif Prior

#Run FIMO for particular motifs
#fimo --oc . --verbosity 1 --thresh 1.0E-4 msn2_all.meme paralogs_LFC_gt_2p0.fasta

#Run exact motif matches

#Extract matches from Roy et al


#Given that data, filter based on 
#size of promoter (200,400,700,1000)
#for FIMO do size of match 

#output just number of matches



#Display +/- data alongside Msn2/4 matches


In [34]:
fc_values

,log2FoldChange_SC,log2FoldChange_msn24del_interaction
0,4.160390,-0.032762
1,4.160390,-0.032762
2,4.160390,-0.032762
3,4.160390,-0.032762
4,2.464130,0.686233
5,2.464130,0.686233
6,2.464130,0.686233
7,2.464130,0.686233
8,-1.806729,0.726361
9,3.793451,-3.095531


In [51]:
fc_values = goi_data_msn2binding.loc[:,['log2FoldChange_SC','log2FoldChange_msn24del_interaction']]
fc_names = ['LFC','LFC Msn24 int']

msn2_promoter_binding_columns = ['MSN2_roy_score', 'MSN4_roy_score']
msn2_promoter_binding_names = ['MSN2_roy', 'MSN4_roy']
msn2_promoter_binding_values =goi_data_msn2binding.loc[:,msn2_promoter_binding_columns]

#want this label to capture paralog pairs eventually
gene_labels = list(goi_data_msn2binding['SC_common_name'])

trace1 = pygo.Heatmap(z = fc_values.values,
                   x= fc_names,
                   y= gene_labels, 
                   showscale = False
                   )

trace2 = pygo.Heatmap(z = msn2_promoter_binding_values.values,
                   #   zmin = -3.0, 
                   #   zmax = 3.0,
                   #text = np.array(joint_topology_text).transpose(),
                   x= msn2_promoter_binding_names,
                   y= gene_labels, 
                   #hoverinfo = 'x+y+text',
                   colorscale = 'Electric',
                   showscale = False)

# trace3 = pygo.Heatmap(z = growth_phenotypes.values,
#                       #text = np.array(topology_text).transpose(),
#                    x=growth_column_names,
#                    y= gene_labels,
#                    colorscale = 'Blackbody',
#                    #hoverinfo = 'x+y+text',
#                    showscale = False)


fig = pytools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
#fig.append_trace(trace3, 1, 3)

#fig['layout'].update(height = 2000)
#fig['layout'].update(height=600, width=600, title='i <3 subplots')

# layout = pygo.Layout(
#     yaxis = {'title': 'Low Paralog / High Paralog'}
# )

#fig = pygo.Figure(data=data, layout=layout)
py.iplot(fig, filename='goi_msn2_heatmap')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [13]:
goi_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'SC_common_name', 'sc_genename',
       'log2FoldChange_SC', 'raw_AS_noNMPP1_YPD_t40_mean_SC', 'pvalue_SC',
       'raw_AS_yesNMPP1_YPD_t40_mean_SC', 'kl_genename', 'log2FoldChange_KL',
       'raw_AS_noNMPP1_YPD_t40_mean_KL', 'raw_AS_yesNMPP1_YPD_t40_mean_KL',
       'pvalue_KL', 'category', 'log2FoldChange_msn24del_interaction',
       'pvalue_msn24del_interaction'],
      dtype='object')

In [ ]:
io_library.SC_common_name_lookup([])